In [1]:
import numpy as np
from scipy.constants import c, epsilon_0 as eps_0, mu_0
import matplotlib.pyplot as plt

In [2]:
# Drude material model
class DrudeMaterial:
    def __init__(self, eps_s, omega_p, gamma):
        self.eps_s = eps_s
        self.omega_p = omega_p
        self.gamma = gamma

    def eps_r(self, omega):
        eps_r = self.eps_s*(1 - self.omega_p**2 / omega / (omega + 1j * self.gamma))
        return eps_r       

In [3]:
# Materials
# Indium Antimonide
eps_s = 15.68
omega_p = 1.860e14 # rad/s
gamma = 1e12 # 1/s
InSb = DrudeMaterial(eps_s,omega_p,gamma)

# Gold
eps_s = 9.1
omega_p = 1.38e16 / np.sqrt(eps_s) # rad/s
gamma = 1.08e14 # 1/s !!! Pocock 2018, check with others'
Au = DrudeMaterial(eps_s,omega_p,gamma);

# Silver
eps_s = 3.3
omega_p = 1.38e16 / np.sqrt(eps_s) # rad/s
gamma = 1.08e14 # 1/s !!! Pocock 2018, check with others'
Au = DrudeMaterial(eps_s,omega_p,gamma);

A sphere with a radius $R << \lambda$
Polarizability

In [ ]:
# Objects
class SWLSphere: # sub-wavelength sphere

    def __init__(self, radius, material):
        self.radius = radius
        self.material = material

    def polarizability(self,omega):
        alpha = 4 * np.pi * self.radius**3 * (self.material.eps_r(omega) - 1) / (self.material.eps_r(omega) + 2) * np.eye(3)
        return alpha

    def resonanceFrequency(self):
        omeagLP = np.sqrt(self.material.eps_s / (self.material.eps_s + 2) * self.material.omega_p**2 - self.material.gamma**2 / 4)
        return omeagLP
        # Calc other parameters: gamma, A, phi
    

In [42]:
radius1 = 10e-9 # [m]
material1 = Au
speher1 = SWLSphere(radius1, material1)
omega_LP = speher1.resonanceFrequency()
print(omega_LP)
lambda_LP = 2 * np.pi * c / omega_LP * 1e9 # [nm]
print(lambda_LP)

4141719541054989.5
454.7994012238319


In [43]:
# Green's functions
# Change this to a class or just have a list of functions?
def GreenEE0(vR, omega):
    # Calculates Green's function j --> E
    R = np.linalg.norm(vR)
    k = omega / c
    exp_term = np.exp(1j * k * R) / (4 * np.pi * R)
    term1 = (1 + 1j / (k * R) - 1 / (k * R)**2) * np.eye(3)
    term2 = (-1 - 3j / (k * R) + 3 / (k * R)**2) / R**2 * np.outer(vR, vR)
    g = exp_term * (term1 + term2)
    return g

In [19]:
# Finite Array of particles
class finiteArray:
    
    def __init__(self, particles, centers):      
        self.centers = centers; # centers[ii,:]: ii'th center: 
        self.particles = particles;

    def couplingMatrix(self, omega):
        # Calculates the coupling matrix from Green's function
        N = self.centers.shape[0]
        M = np.empty((3 * N, 3 * N), dtype=complex)
        for ii in range(N):
            for jj in range(N):
                if ii == jj:
                    M[3*ii:3*ii+3, 3*jj:3*jj+3] = np.zeros((3, 3)) # add self-interaction
                else:
                    M[3*ii:3*ii+3, 3*jj:3*jj+3] = (mu_0 * omega**2 * 
                                                    self.GreenEE(self.centers[ii] - self.centers[jj], omega))
        return M
    
    # Can we give a handle of function as an attribute to a class?
    def GreenEE(self, r, rs, omega):
        # Green's function
        g = GreenEE0(r - rs, omega)
        return g


    def naturalFreqsApprox(self):
        # Approximate complex natural frequencies of the coupled ensemble
        omegaLP = self.particles[0].resonanceFrequency()  # the narrowest resonance among the particles
        gammaLP = self.particles[0].material.gamma / 2
        eigenValues = np.linalg.eig(self.couplingMatrix(omegaLP - 1j * gammaLP))
        # Approximate complex natural frequencies 
        N = np.eigenValues.shape[0]
        return  
    
    def naturalFreqs(self):
        # Finds complex natural frequencies of the coupled ensemble
        # fun = lambda x: np.linalg.det(self.couplingMatrix(x[0] + 1j * x[1]))
        # x0 = self.approximateNM()
        # omegas = fsolve(fun, x0)
        # amplitudes of dipoles from (M-1/alpha)P = 0
        # How to find to P ???


SyntaxError: incomplete input (832338674.py, line 43)